In [1]:
import pandas as pd
import numpy as np
import importlib
import os
from Utils import model_data_engineering
from Utils import preprocessing 
from Utils import plotting

# reload
importlib.reload(model_data_engineering)
importlib.reload(preprocessing)
importlib.reload(plotting)


# importing modules
from Utils.preprocessing import combine_csv_files
from Utils.preprocessing import perform_downsampling
from Utils.plotting import plot
from Utils.featureEngineering import creating_more_features
from sklearn.model_selection import train_test_split


from Utils import Evaluation
importlib.reload(Evaluation)


from Utils import model_data_engineering
importlib.reload(model_data_engineering)

import numpy as np
import os
from sklearn.model_selection import train_test_split


2023-07-24 18:36:47.761956: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-24 18:36:48.733789: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# # # Example usage
# df = combine_csv_files('../raw_data/', '../preprocessed_data/', saveFile=True)
# df.head()

In [ ]:
# Example usage
df = pd.read_csv('../preprocessed_data/combined.csv')

In [ ]:
hourly_df = perform_downsampling(df, freq='1H',aggregation_func='mean')


In [ ]:
x_feature_name = 'Timestamp'
y_feature_name = 'Summe'
title = 'Hourly Plot '
plot(hourly_df,x_feature_name,y_feature_name,title = title,preview=True)

# Custom Model

In [ ]:
features_df = creating_more_features(hourly_df,'Timestamp')
features_df['TimeOfDay'] = features_df['TimeOfDay'].map({'Night': 0, 'Morning': 1, 'Afternoon': 2, 'Evening': 3})


features_df.head()

In [ ]:

holiday_df = features_df[features_df['Holiday']==1]



weekned_df = features_df[features_df['Weekend']==1]


Night_df = features_df[features_df['TimeOfDay']==0]
Morning_df = features_df[features_df['TimeOfDay']==1]
evening_Df = features_df[features_df['TimeOfDay']==3]
afternoon_df = features_df[features_df['TimeOfDay']==2]
monday_df = features_df[features_df['DayOfWeek']==0]
tuesday_df = features_df[features_df['DayOfWeek']==1]
wedness_df = features_df[features_df['DayOfWeek']==2]
thurday_df = features_df[features_df['DayOfWeek']==3]
friday_df = features_df[features_df['DayOfWeek']==4]


In [ ]:
import plotly.graph_objects as go  # Library for creating interactive plots
import numpy as np
import os
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=features_df['Timestamp'],
        y=features_df['Summe'],
        name=y_feature_name,
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=holiday_df['Timestamp'],
        y=holiday_df['Summe'],
        name='Public Holiday',
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=weekned_df['Timestamp'],
        y=weekned_df['Summe'],
        name='Saturday+Sunday',
        mode='lines+markers'
    ))

fig.add_trace(
    go.Scatter(
        x=Night_df['Timestamp'],
        y=Night_df['Summe'],
        name='Night Time',
        mode='lines+markers'
    ))

fig.add_trace(
    go.Scatter(
        x=Morning_df['Timestamp'],
        y=Morning_df['Summe'],
        name='Morning Time',
        mode='lines+markers'
    ))

fig.add_trace(
    go.Scatter(
        x=evening_Df['Timestamp'],
        y=evening_Df['Summe'],
        name='EveningTime',
        mode='lines+markers'
    ))

fig.add_trace(
    go.Scatter(
        x=afternoon_df['Timestamp'],
        y=afternoon_df['Summe'],
        name='AfterNoon',
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=monday_df['Timestamp'],
        y=monday_df['Summe'],
        name='Monday',
        mode='lines+markers'
    ))


fig.add_trace(
    go.Scatter(
        x=tuesday_df['Timestamp'],
        y=tuesday_df['Summe'],
        name='Tuesday',
        mode='lines+markers'
    ))

fig.add_trace(
    go.Scatter(
        x=wedness_df['Timestamp'],
        y=wedness_df['Summe'],
        name='Wednessday',
        mode='lines+markers'
    ))

fig.add_trace(
    go.Scatter(
        x=thurday_df['Timestamp'],
        y=thurday_df['Summe'],
        name='Thursday',
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=friday_df['Timestamp'],
        y=friday_df['Summe'],
        name='Friday',
        mode='lines+markers'
    ))



# Update xaxis properties
# The x-axis and y-axis titles are updated using the update_xaxes() and update_yaxes() methods of the figure object.
fig.update_xaxes(title_text='Date')

# Update yaxis properties
fig.update_yaxes(title_text=y_feature_name)

# Update title and height
# The layout of the figure is updated using the update_layout() method. The title, height, and width of the plot are set.
fig.update_layout(
    title=f'{title}',
    height=500,
    width=1200
)

fig.write_html('DetailVisualization_of_hourlyData.html')


In [ ]:
features_df[features_df['Hour']==1]

In [ ]:
train , test =  train_test_split(features_df,train_size=0.8,shuffle=False)

In [ ]:
train

In [2]:
import numpy as np

def give_me_historical_values(train, test):
    test['Predicted_mean'] = np.float64('nan')
    test['Predicted_median'] = np.float64('nan')
    test['predicted_monthly_mean'] = np.float64('nan') 
    test['predicted_monthly_median'] = np.float64('nan') 
    test['predicted_base_on_lag1'] =  np.float64('nan')
    for index, row in test.iterrows():
        hour = row['Hour']
        dayofWeek = row['DayOfWeek']
        weekend = row['Weekend']
        timeOfDay = row['TimeOfDay']
        day = row['Day']
        month = row['Month']
        Year = row['Year']

        # print(f'Hour: {hour}\nDayOfWeek: {dayofWeek}\nWeekend: {weekend}\nTimeOfDay: {timeOfDay}\nDay: {day}\nMonth:{month}\nYear:{Year}')
        train_df = train[
            (train['Hour'] == hour) &
            (train['DayOfWeek'] == dayofWeek) &
            (train['Weekend'] == weekend) &
            (train['TimeOfDay'] == timeOfDay) &
            (train['Day'] == day) 
        ]
        
        last_consective_day = train[
            (train['Hour'] == hour) &
            (train['DayOfWeek'] == dayofWeek) &
            (train['Weekend'] == weekend) &
            (train['TimeOfDay'] == timeOfDay)
        ]
    
        
        
        train_df_monthly = train[
            (train['Hour'] == hour) &
            (train['DayOfWeek'] == dayofWeek) &
            (train['Weekend'] == weekend) &
            (train['TimeOfDay'] == timeOfDay) &
            (train['Month'] == month) 
        ]
        # display(last_consective_day)
        
        hourly_values_mean = train_df['Summe'].mean()
        hourly_values_median = train_df['Summe'].median()
        
        monthly_values_mean = train_df_monthly['Summe'].mean()
        monthly_values_median = train_df_monthly['Summe'].median()

        test.at[index, 'Predicted_mean'] = hourly_values_mean
        test.at[index, 'Predicted_median'] = hourly_values_median

        test.at[index, 'predicted_monthly_mean'] = monthly_values_mean
        test.at[index, 'predicted_monthly_median'] = monthly_values_median
        
        test.at[index, 'predicted_base_on_lag1'] = last_consective_day.iloc[-1].Summe
        
    return test 


In [3]:

result_df= give_me_historical_values(train,test)
result_df    


NameError: name 'train' is not defined

In [214]:
import plotly.graph_objects as go  # Library for creating interactive plots
import numpy as np
import os
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['Summe'],
        name='Actual',
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['Predicted_mean'],
        name='Predicted by Mean',
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['Predicted_median'],
        name='Predicted by Medain',
        mode='lines+markers'
    ))

fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['predicted_monthly_mean'],
        name='added Monthly-Predicted by Median',
        mode='lines+markers'
    ))
fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['predicted_monthly_median'],
        name='added Monthly-Predicted by Mean',
        mode='lines+markers'
    ))

fig.add_trace(
    go.Scatter(
        x=result_df['Timestamp'],
        y=result_df['predicted_base_on_lag1'],
        name='lag-1',
        mode='lines+markers'
    ))







# Update xaxis properties
# The x-axis and y-axis titles are updated using the update_xaxes() and update_yaxes() methods of the figure object.
fig.update_xaxes(title_text='Date')

# Update yaxis properties
fig.update_yaxes(title_text='Summe')

# Update title and height
# The layout of the figure is updated using the update_layout() method. The title, height, and width of the plot are set.
fig.update_layout(
    title=f'{title}',
    height=500,
    width=1200
)


fig.write_html('Results.html')


In [ ]:
hourly_values

In [215]:
import numpy as np

def Evaluation(actual_values, predicted_values):
    """
    Calculate Mean Absolute Error (MAE).

    Parameters:
        actual_values (array-like): Array or pandas Series containing the actual values.
        predicted_values (array-like): Array or pandas Series containing the predicted values.

    Returns:
        float: The Mean Absolute Error (MAE).
    """
    mae = np.mean(np.abs(actual_values - predicted_values))
    mape = np.mean(np.abs((actual_values - predicted_values) / actual_values)) * 100


    return mae , mape

print('by Mean',Evaluation(value['Summe'],value['Predicted_mean']))
print('by Medain',Evaluation(value['Summe'],value['Predicted_median']))
print('by Mean(added Monthly)',Evaluation(value['Summe'],value['predicted_monthly_mean']))
print('by Mean(added Monthly)',Evaluation(value['Summe'],value['predicted_monthly_median']))
print('by lag-1',Evaluation(value['Summe'],value['predicted_base_on_lag1']))



by Mean (57.09417163213578, 118.38122606685366)
by Medain (57.09417163213578, 118.38122606685366)
by Mean(added Monthly) (52.457764085828856, 111.47785930663932)
by Mean(added Monthly) (39.49098555227035, 50.86659395817461)
by lag-1 (69.09466292051762, 165.3488817240063)
